# HDA - Project 3

This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- folder: directory name where '.mat' files are stored;
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 1
label = 6   # default for task B1
folder = "../data/full/"
window_size = 15
stride = 5

In [2]:
import preprocessing
import models
import numpy as np
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In the following cell, we make use of some functions of Keras which have been removed, but of which the code is still available at https://github.com/keras-team/keras/commit/a56b1a55182acf061b1eb2e2c86b48193a0e88f7. These are used to evaulate the f1 score during training on batches of data: this is only an approximation though, which is the reason why they have been removed.

In [3]:
import keras.backend as K
from sklearn.metrics import f1_score


def precision(y_true, y_pred): 
    """Precision metric.
    
    Only computes a batch-wise average of precision. 
    Computes the precision, a metric for multi-label classification of 
    how many selected items are relevant. 
    """ 
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon()) 

    return precision


def recall(y_true, y_pred): 
    """Recall metric. 
    
    Only computes a batch-wise average of recall. 
    Computes the recall, a metric for multi-label classification of 
    how many relevant items are selected. 
    """ 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) 
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon()) 
 
    return recall

def fbeta_score(y_true, y_pred, beta=1): 
    """Computes the F score. 

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0 

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon()) 

    return fbeta_score 


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.
    
    Here it is only computed as a batch-wise average, not globally.
    """ 

    return fbeta_score(y_true, y_pred, beta=1) 

## Classification with null class
### Preprocessing

In [4]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadDataMultiple(label=label,
                                                                                                        folder=folder,
                                                                                                        window_size=window_size,
                                                                                                        stride=stride,
                                                                                                        make_binary=False,
                                                                                                        null_class=True,
                                                                                                        print_info=True)


Processing data from subject 1

Session shapes:
ADL1:   (45810, 110)
ADL2:   (28996, 110)
ADL3:   (30167, 110)
ADL4:   (30228, 110)
ADL5:   (27308, 110)
Drill:  (52152, 110)

Features: 110 
Classes: 18 
Fraction of labels:   [0.66779111 0.01365242 0.01473443 0.01120199 0.02020813 0.02074913
 0.01183846 0.01613468 0.01307959 0.01377972 0.01584826 0.01225217
 0.02132196 0.02199026 0.01152022 0.01724851 0.07889126 0.01775769]

Features: 110 
Classes: 18 
Fraction of labels:   [0.77548892 0.00764885 0.00591047 0.00808344 0.01425467 0.01156019
 0.00365059 0.01833985 0.00990874 0.0065189  0.00808344 0.00582355
 0.01642764 0.01399392 0.00295524 0.01521078 0.06996958 0.00617123]

Processing data from subject 2

Session shapes:
ADL1:   (38733, 110)
ADL2:   (26824, 110)
ADL3:   (31242, 110)
ADL4:   (29723, 110)
ADL5:   (27997, 110)
Drill:  (49009, 110)

Features: 110 
Classes: 18 
Fraction of labels:   [0.6284852  0.01879351 0.01416372 0.00823073 0.0201996  0.02157138
 0.00956823 0.02249734 0.0

### Model

In [ ]:
detection_model = models.MotionDetection((window_size, n_features), n_classes)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy", fmeasure])

checkpointer = ModelCheckpoint(filepath='./weights_1.hdf5', verbose=1, save_best_only=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 15, 110)           440       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 36)             43596     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5, 36)             0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2, 36)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 600)            1528800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 600)               2882400   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               307712    
__________

### Training

In [ ]:
detection_model.fit(x = X_train, 
                    y = Y_train, 
                    epochs = 20, 
                    batch_size = 16,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, Y_test),
                    class_weight=class_weights)

Train on 114295 samples, validate on 41979 samples
Epoch 1/20
 47056/114295 [===========>..................] - ETA: 2:10 - loss: 0.9074 - acc: 0.7332 - fmeasure: 0.7275

### Evaluation

In [ ]:
Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))

In [ ]:
detection_model_best = load_model('./weights_1.hdf5')

Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))

## Classification without null class
### Preprocessing

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=False,
                                                                                                print_info=True)

### Model

In [ ]:
detection_model = models.MotionDetection((window_size, n_features), n_classes)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy", fmeasure])

checkpointer = ModelCheckpoint(filepath='./weights_2.hdf5', verbose=1, save_best_only=True)

### Training

In [ ]:
detection_model.fit(x = X_train, 
                    y = Y_train, 
                    epochs = 20, 
                    batch_size = 16,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, Y_test),
                    class_weight=class_weights)

### Evaluation

In [ ]:
Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))

In [ ]:
detection_model_best = load_model('./weights_2.hdf5')

Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))

## Activity detection (binary classification)
### Preprocessing

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=True,
                                                                                                null_class=True,
                                                                                                print_info=True)

### Model

In [ ]:
detection_model = models.MotionDetection((window_size, n_features), n_classes)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy", fmeasure])

checkpointer = ModelCheckpoint(filepath='./weights_3.hdf5', verbose=1, save_best_only=True)

In [ ]:
detection_model.fit(x = X_train, 
                    y = Y_train, 
                    epochs = 20, 
                    batch_size = 16,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, Y_test),
                    class_weight=class_weights)

### Evaluation

In [ ]:
Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))

In [ ]:
detection_model_best = load_model('./weights_3.hdf5')

Y_pred = detection_model.predict(X_test)
Y_pred = np.argmax(Y_pred, 1)

print(classification_report(Y_test, to_categorical(Y_pred)))